# Interactive visualization of the loaded graph

## set up environment

Load the library dependencies

In [1]:
import json
import pathlib

import networkx as nx
import pyvis
import watermark

Optional: filter the less useful warnings

In [2]:
#warnings.filterwarnings("ignore")

Show a watermark of the OS, hardware, language environment, and dependent library versions.

In [3]:
%load_ext watermark
%watermark
%watermark --iversions

Last updated: 2025-02-24T11:04:55.760231-08:00

Python implementation: CPython
Python version       : 3.13.2
IPython version      : 8.32.0

Compiler    : Clang 16.0.0 (clang-1600.0.26.6)
OS          : Darwin
Release     : 23.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

networkx : 3.4.2
json     : 2.0.9
pyvis    : 0.3.2
watermark: 2.5.0



## visualize results

In [4]:
MIN_CLIQUE_SIZE: int = 8

def render_graph (
    graph: nx.DiGraph,
    ) -> pyvis.network.Network:
    """
    Generate a `PyVis` network visualization from the DiGraph
    """
    net: pyvis.network.Network = pyvis.network.Network(
        notebook = True,
        cdn_resources = "in_line",
        directed = True,
        height = "900px", 
        width = "100%",
    )

    # filter to only show the usable subgraphs
    usable: set = set()
    
    for clique in nx.weakly_connected_components(graph):
        if len(clique) >= MIN_CLIQUE_SIZE:
            usable.update(clique)

    for node_id in (set(graph.nodes) - usable):
        if node_id not in usable:
            graph.remove_node(node_id)

    # calculate the _betweenness centrality_ measures for each country
    for node_id, rank in nx.betweenness_centrality(graph, weight = "weight").items():
        graph.nodes[node_id]["value"] = rank

    
    # load the network and adjust attributes for visualization
    net.from_nx(graph)

    for node in net.nodes:
        node["label"] = node["name"]

        if node["kind"] == "data":
            node["color"] = "#F690EB"
        else:
            node["color"] = "#BDC561"

        if node["type"] == "ftm:Person":
            node["shape"] = "square"
        else:
            node["shape"] = "dot"

    for edge in net.edges:
        #edge["value"] = edge["width"]
        pass

    return net

In [5]:
graph: nx.DiGraph = nx.DiGraph() # placeholder

graph_file: pathlib.Path = pathlib.Path("graph.json")

with open(graph_file, "r", encoding = "utf-8") as fp:
    graph = nx.node_link_graph(
        json.load(fp),
        edges = "edges",
    )

In [6]:
net: pyvis.network.Network = render_graph(graph)

net.force_atlas_2based()
net.toggle_physics(True)

net.prep_notebook()
net.show_buttons(filter_ = ["physics"])

net.show("networkx-pyvis.html")

networkx-pyvis.html


Note: to see the final graph, load the HTML file `network-pyvis.html` in your browser.